In [ ]:
import numpy as np
import scanpy as sc
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
print(sys.executable)

In [ ]:
h.print_main_versions()

In [ ]:
#env: scArches_env

In [ ]:
pre = "H22"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)
_,base_table_path_H21,_,_ = h.return_local_paths(drive = drive,pre = "H21",add_path = False)

In [ ]:
#human
species = 'human'
warnings.filterwarnings("ignore")
IDs = h.return_cellranger_IDs(species)
adata_all_human = h.read_bg_corrected_counts(drive,cellranger_IDs = IDs,pre_bg_correction = 'H05_01', species = species)

In [ ]:
anno = pd.read_csv((os.path.join(base_table_path_H21,'H21_'  + species + '_anno_QC.csv')),index_col=0)

In [ ]:
adata_QC = h.add_anno_to_adata(adata=adata_all_human,
                  anno_df=anno,
                  anno_columns_to_add=anno.columns,
                  rsuffix='_R')

In [ ]:
adata_QC,nr_removed_cells,nr_removed_genes = h.standard_scanpy_filter(adata_QC,min_genes_per_cell=1,min_cells_per_gene=50)

print('nr_removed_cells : ' + str(nr_removed_cells))
print('nr_removed_genes : ' + str(nr_removed_genes))

In [ ]:
h.prepare_umap(adata_QC)

In [ ]:
for color in ['doublet_QC','basic_QC','QC_summary','louvain_res_7','timepoint','individual','scDblFinder.class2','dblt_doubldetect_guess2','dblt_scrublet_predicted2']:
    h.umap_plot(adata = adata_QC,
            color = color,
            base_plots_path = base_plots_path,
            plot_name = pre + '_umap_human_' + color +  '_.pdf',
            show_in_nb = True,
            frameon=False)

In [ ]:
#add initial celltype annotation from celltypist or azimuth

In [ ]:
path_s0115_cellano_azimuth =  os.path.join('F:\\monkey_IZI\\analysisR','s0115_cellano_azimuth.txt')
s0115_cellano_azimuth = pd.read_csv(path_s0115_cellano_azimuth,delimiter = "\t",index_col =0)

In [ ]:
#human
species = 'human'

In [ ]:
s0115_cellano_azimuth_human = s0115_cellano_azimuth[s0115_cellano_azimuth['species'] == 'human']
adata_QC = h.add_anno_to_adata(adata=adata_QC,
                    anno_df=s0115_cellano_azimuth_human,
                    anno_columns_to_add=s0115_cellano_azimuth_human.columns,rsuffix='_RR')

In [ ]:
initial_ct_columns = ['predicted.celltype.l1','predicted.celltype.l2','predicted.celltype.l3','Healty_WholeBlood_maj','COVID19_PBMC_maj']

In [ ]:
for color in initial_ct_columns:
    h.umap_plot(adata = adata_QC,
            color = color,
            base_plots_path = base_plots_path,
            plot_name = pre + '_umap_human_' + color +  '_.pdf',
            show_in_nb = True,
            frameon=False)

In [ ]:
for anno_column in initial_ct_columns:
    _ = h.distrubtion_per_obs_column(adata_QC,
                           obs_column_group=anno_column,
                           obs_column_cat='QC_summary',
                           normalize = True,
                           sort_by ='passed_QC',
                           ascending = True,
                            save=os.path.join(base_table_path,pre + '_' + anno_column + '_QC_overview_perc.csv'))
    _ = h.distrubtion_per_obs_column(adata_QC,
                           obs_column_group=anno_column,
                           obs_column_cat='QC_summary',
                           normalize = False,
                           sort_by ='passed_QC',
                           ascending = True,
                            save=os.path.join(base_table_path,pre + '_' + anno_column + '_QC_overview_abs.csv'))

In [ ]:
initial_ct_columns

In [ ]:
df_azimuth_lv1_perc = pd.read_csv(os.path.join(base_table_path,pre + '_' + 'predicted.celltype.l1' + '_QC_overview_perc.csv'),index_col=0)

df_azimuth_lv2_perc = pd.read_csv(os.path.join(base_table_path,pre + '_' + 'predicted.celltype.l2' + '_QC_overview_perc.csv'),index_col=0)

df_azimuth_lv3_perc = pd.read_csv(os.path.join(base_table_path,pre + '_' + 'predicted.celltype.l3' + '_QC_overview_perc.csv'),index_col=0)

df_celltypist_H_WB_maj_perc = pd.read_csv(os.path.join(base_table_path,pre + '_' + 'Healty_WholeBlood_maj' + '_QC_overview_perc.csv'),index_col=0)

df_celltypist_COV_PBMC_maj_perc = pd.read_csv(os.path.join(base_table_path,pre + '_' + 'COVID19_PBMC_maj' + '_QC_overview_perc.csv'),index_col=0)

In [ ]:
df_azimuth_lv1_perc

In [ ]:
adata_QC.obs['predicted.celltype.l1'].value_counts(ascending=True)

In [ ]:
df_azimuth_lv2_perc

In [ ]:
adata_QC.obs['predicted.celltype.l2'].value_counts(ascending=True)

In [ ]:
df_azimuth_lv3_perc

In [ ]:
adata_QC.obs['predicted.celltype.l3'].value_counts(ascending=True)

In [ ]:
df_celltypist_H_WB_maj_perc

In [ ]:
adata_QC.obs['Healty_WholeBlood_maj'].value_counts(ascending=True)

In [ ]:
df_celltypist_COV_PBMC_maj_perc

In [ ]:
adata_QC.obs['COVID19_PBMC_maj'].value_counts(ascending=True)

In [ ]:
#separete anndata objects according to result of cluster-based QC

In [ ]:
adata_QC_in = adata_QC[adata_QC.obs['QC_summary'] == 'passed_QC']
adata_QC_out = adata_QC[~(adata_QC.obs['QC_summary'] == 'passed_QC')]

In [ ]:
#compare distribution of initial cell type annotations

In [ ]:
for anno_column in initial_ct_columns:
    h.make_pie_plot_QC(adata_QC_out.obs[anno_column].value_counts(),title =anno_column,
                   save=os.path.join(base_plots_path,pre + '_pie_QC_out' + anno_column + '.pdf'),
                   show=True)
    h.make_pie_plot_QC(adata_QC_in.obs[anno_column].value_counts(),title =anno_column,
                       save=os.path.join(base_plots_path,pre + '_pie_QC_in' + anno_column + '.pdf'),
                       show=True)

In [ ]:
#adata_QC.write_h5ad(os.path.join(base_anndata_objects,pre + '_QC_human_all.h5ad'))
#adata_QC_in.write_h5ad(os.path.join(base_anndata_objects,pre + '_QC_human_passedQC.h5ad'))
#adata_QC_out.write_h5ad(os.path.join(base_anndata_objects,pre + '_QC_human_failedQC.h5ad'))

# Save session

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()